## Get IMSI and ICCID from MSISDN list

In [1]:
import pandas as pd
import numpy as np
# Import datetime from the datetime module
from datetime import datetime

### MSISDN List

In [2]:
msisdn_list = pd.read_csv('data/MSISDN_PAGSEGURO.txt')
msisdn_list = msisdn_list.applymap(str)
msisdn_list.rename(columns = {'MSISDN': 'msisdn'}, inplace = True)                
msisdn_list.shape

(15053, 1)

In [3]:
msisdn_list.head()

,msisdn
0,5511997361390
1,5511942788371
2,5511942851508
3,5511933952833
4,5511933910161


In [4]:
msisdn_list.dtypes

msisdn    object
dtype: object

### Sincard Data

In [5]:
def getDataFromKite(csvFile):
    df = pd.read_csv(csvFile, sep=';', low_memory=False)
    df = df[['icc', 'imsi', 'msisdn']]
    df = df.applymap(str) # change all columns to string type
    df = df.applymap(lambda x: x.lstrip('=""').rstrip('"')) # remove unnecessary characteres
    
    return df

In [6]:
%%time
simDF_20 = getDataFromKite('data/SimInventoryExport_2021-03-24T21.07.31Z_1p54bnb-3hrww7e2lmb-605baa139fb2.0.csv')
simDF_20.shape

CPU times: user 38.6 s, sys: 5.75 s, total: 44.3 s
Wall time: 44.5 s


(1000000, 3)

In [7]:
# simDF_20.head()

In [8]:
%%time
simDF_21 = getDataFromKite('data/SimInventoryExport_2021-03-24T21.07.31Z_1p54bnb-3hrww7e2lmb-605baa139fb2.1.csv')
simDF_21.shape

CPU times: user 36.8 s, sys: 4.53 s, total: 41.4 s
Wall time: 43.3 s


(953216, 3)

In [9]:
# simDF_21.head()

### Inner Join

https://www.datacamp.com/community/tutorials/joining-dataframes-pandas?utm_source=adwords_ppc&utm_campaignid=1455363063&utm_adgroupid=65083631748&utm_device=c&utm_keyword=&utm_matchtype=b&utm_network=g&utm_adpostion=&utm_creative=278443377095&utm_targetid=aud-299261629574:dsa-429603003980&utm_loc_interest_ms=&utm_loc_physical_ms=1001773&gclid=CjwKCAjw6fCCBhBNEiwAem5SO0NwQmPm3bLhWMfwnkay1aW7Akt8Iu4DxHjmjULI25wxeUV4QUEHzxoCV-kQAvD_BwE

In [10]:
msisdn_list.shape, simDF_20.shape, simDF_21.shape

((15053, 1), (1000000, 3), (953216, 3))

In [11]:
%%time
df_inner_20 = pd.merge(msisdn_list, simDF_20, on='msisdn', how='inner')
df_inner_20.shape

CPU times: user 715 ms, sys: 39.3 ms, total: 754 ms
Wall time: 801 ms


(45, 3)

In [12]:
%%time
df_inner_21 = pd.merge(msisdn_list, simDF_21, on='msisdn', how='inner')
df_inner_21.shape

CPU times: user 671 ms, sys: 9 ms, total: 680 ms
Wall time: 727 ms


(15008, 3)

In [13]:
df_inner_full = pd.concat([df_inner_20, df_inner_21])
df_inner_full.shape

(15053, 3)

In [14]:
msisdn_list.shape[0] == df_inner_full.shape[0] == (df_inner_20.shape[0] + df_inner_21.shape[0])

True

In [15]:
# df_inner_full.head()

In [17]:
df_inner_full['msisdn'].isna().sum(), df_inner_full['msisdn'].isnull().sum()

(0, 0)

### Output File

In [18]:
def getCurrentDate():
    local_dt = datetime.now()
    return str(local_dt.year) +'.'+ str(local_dt.month) + '.' + str(local_dt.day) + '_' + str(local_dt.hour) + 'h' + str(local_dt.minute)

In [20]:
filename = 'data/result/msisdn_iccid_result' + getCurrentDate() + '.xlsx'
df_inner_full.to_excel(filename, sheet_name='data')